#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("D:\LangChain\DataIngestion\speech.txt")
documents = loader.load()


In [3]:
text_splitter  = CharacterTextSplitter(chunk_size = 200,chunk_overlap = 20)
docs = text_splitter.split_documents(documents)

Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 347, which is longer than the specified 200
Created a chunk of size 668, which is longer than the specified 200
Created a chunk of size 982, which is longer than the specified 200
Created a chunk of size 789, which is longer than the specified 200


In [4]:
docs

[Document(metadata={'source': 'D:\\LangChain\\DataIngestion\\speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'D:\\LangChain\\DataIngestion\\speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

In [5]:
embeddings = OllamaEmbeddings(model="gemma:2b")
db = FAISS.from_documents(docs,embeddings)
db

C:\Users\aryan\AppData\Local\Temp\ipykernel_36416\3469876571.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="gemma:2b")


In [7]:
## querying
query = " What does the speaker believe is the main reason the United States should enter the war ? "
docs = db.similarity_search(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [8]:
retriver = db.as_retriever()
docs = retriver.invoke(query)
docs

[Document(id='556e578f-f6f9-4e82-a60d-4649c28c50a6', metadata={'source': 'D:\\LangChain\\DataIngestion\\speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='b86c0810-80b9-49f9-baac-9786d9e372cf', metadata={'source': 'D:\\LangChain\\DataIngestion\\speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is m

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [9]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='556e578f-f6f9-4e82-a60d-4649c28c50a6', metadata={'source': 'D:\\LangChain\\DataIngestion\\speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
  np.float32(4992.255)),
 (Document(id='b86c0810-80b9-49f9-baac-9786d9e372cf', metadata={'source': 'D:\\LangChain\\DataIngestion\\speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the 

In [10]:
embeddins_vector = embeddings.embed_query(query)

In [11]:
db.similarity_search_by_vector(embeddins_vector)

[Document(id='556e578f-f6f9-4e82-a60d-4649c28c50a6', metadata={'source': 'D:\\LangChain\\DataIngestion\\speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='b86c0810-80b9-49f9-baac-9786d9e372cf', metadata={'source': 'D:\\LangChain\\DataIngestion\\speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is m

In [12]:
## Saving and loading 
db.save_local("faiss_index")

In [14]:
new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

In [17]:
new_db.similarity_search(query)

[Document(id='556e578f-f6f9-4e82-a60d-4649c28c50a6', metadata={'source': 'D:\\LangChain\\DataIngestion\\speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='b86c0810-80b9-49f9-baac-9786d9e372cf', metadata={'source': 'D:\\LangChain\\DataIngestion\\speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is m